## In-context learning with Falcon 7B Instruct

In this lab you will use the Falcon 7B Instruct model taht you deployed in the prerequisities section and see examples of in-context learning using this deployed model.

#### Imports 

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker import get_execution_role
import sagemaker
import logging
import boto3
import time
import json

#### Setup essentials 

In [ ]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [ ]:
logger.info(f'Using sagemaker=={sagemaker.__version__}')
logger.info(f'Using boto3=={boto3.__version__}')


In [ ]:
# set up roles and clients 
client = boto3.client('sagemaker-runtime')
ROLE = get_execution_role()

#### I. Get the deployed model instance
Replace the variable `endpoint_name` with the name of the endpoint you deployed for the Falcon 7V model. You can retrieve that by navigating on the left side through Home -> Deployments -> Endpoints. Select the ednpoint that starts with "hf-llm-falcon-7b-instruct-bf16-" and replace its value with the variable `endpoint_name`

In [ ]:
# Use the existing endpoint name
endpoint_name = "<endpoint name>"  # Replace with your endpoint name

# Create a SageMaker predictor object
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)
CONTENT_TYPE = 'application/json'

predictor.endpoint 

#### II. Invoke the SageMaker endpoint to test the deployed model for natural language understanding (NLU) and natural language generation (NLG) tasks

In [ ]:
context = """
Customer: Hi there, I'm having a problem with my iPhone.
Agent: Hi! I'm sorry to hear that. What's happening?
Customer: The phone is not charging properly, and the battery seems to be draining very quickly. I've tried different charging cables and power adapters, but the issue persists.
Agent: Hmm, that's not good. Let's try some troubleshooting steps. Can you go to Settings, then Battery, and see if there are any apps that are using up a lot of battery life?
Customer: Yes, there are some apps that are using up a lot of battery.
Agent: Okay, try force quitting those apps by swiping up from the bottom of the screen and then swiping up on the app to close it.
Customer: I did that, but the issue is still there.
Agent: Alright, let's try resetting your iPhone's settings to their default values. This won't delete any of your data. Go to Settings, then General, then Reset, and then choose Reset All Settings.
Customer: Okay, I did that. What's next?
Agent: Now, let's try restarting your iPhone. Press and hold the power button until you see the "slide to power off" option. Slide to power off, wait a few seconds, and then turn your iPhone back on.
Customer: Alright, I restarted it, but it's still not charging properly.
Agent: I see. It looks like we need to run a diagnostic test on your iPhone. Please visit the nearest Apple Store or authorized service provider to get your iPhone checked out.
Customer: Do I need to make an appointment?
Agent: Yes, it's always best to make an appointment beforehand so you don't have to wait in line. You can make an appointment online or by calling the Apple Store or authorized service provider.
Customer: Okay, will I have to pay for the repairs?
Agent: That depends on whether your iPhone is covered under warranty or not. If it is, you won't have to pay anything. However, if it's not covered under warranty, you will have to pay for the repairs.
Customer: How long will it take to get my iPhone back?
Agent: It depends on the severity of the issue, but it usually takes 1-2 business days.
Customer: Can I track the repair status online?
Agent: Yes, you can track the repair status online or by calling the Apple Store or authorized service provider.
Customer: Alright, thanks for your help.
Agent: No problem, happy to help. Is there anything else I can assist you with?
Customer: No, that's all for now.
Agent: Alright, have a great day and good luck with your iPhone!
"""

Generation configuration 

In [ ]:
MAX_LENGTH = 1024
NUM_RETURN_SEQUENCES = 1
TOP_P = 0.9
TOP_K = 10
DO_SAMPLE = True 

#### A. Text Summarization 

In [ ]:
query = 'write a summary'

In [ ]:
prompt = f'{context}\n{query}'

In [ ]:
payload = {'inputs': prompt, 
           'parameters': {
               'max_new_tokens': MAX_LENGTH, 
               'num_return_sequences': NUM_RETURN_SEQUENCES,
               'top_p': TOP_P,
               'top_k': TOP_K,
               'do_sample': DO_SAMPLE
           }
          }

In [ ]:
payload = json.dumps(payload).encode('utf-8')

In [ ]:
response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType=CONTENT_TYPE, 
                                  Body=payload)

In [ ]:
model_predictions = json.loads(response["Body"].read().decode('utf-8'))
generated_text = model_predictions[0]['generated_text']
print(f'Response: {generated_text}')

#### B. Abstractive Question Answering 

##### Q1

In [ ]:
query = 'What troubleshooting steps were suggested to the customer to fix their iPhone charging issue?'

In [ ]:
prompt = f'{context}\n{query}'

In [ ]:
payload = {'inputs': prompt, 
           'parameters': {
               'max_new_tokens': MAX_LENGTH, 
               'num_return_sequences': NUM_RETURN_SEQUENCES,
               'top_p': TOP_P,
               'top_k': TOP_K,
               'do_sample': DO_SAMPLE
           }
          }

In [ ]:
payload = json.dumps(payload).encode('utf-8')

In [ ]:
response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType=CONTENT_TYPE, 
                                  Body=payload)

In [ ]:
model_predictions = json.loads(response["Body"].read().decode('utf-8'))
generated_text = model_predictions[0]['generated_text']
print(f'Response: {generated_text}')

Q2

In [ ]:
query = 'Was resetting the iPhone to its default settings able to solve the customer\'s problems?'

In [ ]:
prompt = f'{context}\n{query}'

In [ ]:
payload = {'inputs': prompt, 
           'parameters': {
               'max_new_tokens': MAX_LENGTH, 
               'num_return_sequences': NUM_RETURN_SEQUENCES,
               'top_p': TOP_P,
               'top_k': TOP_K,
               'do_sample': DO_SAMPLE
           }
          }

In [ ]:
payload = json.dumps(payload).encode('utf-8')

In [ ]:
response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType=CONTENT_TYPE, 
                                  Body=payload)

In [ ]:
model_predictions = json.loads(response["Body"].read().decode('utf-8'))
generated_text = model_predictions[0]['generated_text']
print(f'Response: {generated_text}')

Q3

In [ ]:
query = 'What steps can the customer take to make an appointment at the nearest Apple Store or authorized service provider for iPhone repair?'

In [ ]:
prompt = f'{context}\n{query}'

In [ ]:
payload = {'inputs': prompt, 
           'parameters': {
               'max_new_tokens': MAX_LENGTH, 
               'num_return_sequences': NUM_RETURN_SEQUENCES,
               'top_p': TOP_P,
               'top_k': TOP_K,
               'do_sample': DO_SAMPLE
           }
          }

In [ ]:
payload = json.dumps(payload).encode('utf-8')

In [ ]:
response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType=CONTENT_TYPE, 
                                  Body=payload)

In [ ]:
model_predictions = json.loads(response["Body"].read().decode('utf-8'))
generated_text = model_predictions[0]['generated_text']
print(f'Response: {generated_text}')

#### C. Sentiment Analysis

In [ ]:
query = 'What is the overall sentiment and sentiment score of the conversation between the customer and the agent'

In [ ]:
prompt = f'{context}\n{query}'

In [ ]:
payload = {'inputs': prompt, 
           'parameters': {
               'max_new_tokens': MAX_LENGTH, 
               'num_return_sequences': NUM_RETURN_SEQUENCES,
               'top_p': TOP_P,
               'top_k': TOP_K,
               'do_sample': DO_SAMPLE
           }
          }

In [ ]:
payload = json.dumps(payload).encode('utf-8')

In [ ]:
response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType=CONTENT_TYPE, 
                                  Body=payload)

In [ ]:
model_predictions = json.loads(response["Body"].read().decode('utf-8'))
generated_text = model_predictions[0]['generated_text']
print(f'Response: {generated_text}')

In [ ]:
sentiment = generated_text

#### D. Sentiment Phrase Extraction

In [ ]:
query = f'identify any specific words, phrases, or context that influenced the {sentiment} sentiment'

In [ ]:
prompt = f'{context}\n{query}'

In [ ]:
payload = {'inputs': prompt, 
           'parameters': {
               'max_new_tokens': MAX_LENGTH, 
               'num_return_sequences': NUM_RETURN_SEQUENCES,
               'top_p': TOP_P,
               'top_k': TOP_K,
               'do_sample': DO_SAMPLE
           }
          }

In [ ]:
payload = json.dumps(payload).encode('utf-8')

In [ ]:
response = client.invoke_endpoint(EndpointName=endpoint_name, 
                                  ContentType=CONTENT_TYPE, 
                                  Body=payload)

In [ ]:
model_predictions = json.loads(response["Body"].read().decode('utf-8'))
generated_text = model_predictions[0]['generated_text']
print(f'Response: {generated_text}')